In [ ]:
import os
from typing import List

import numpy as np
import pandas as pd
import torch
from rdkit.Chem.rdmolfiles import MolFromSmarts
from tokenizers import Tokenizer

from src.data.components.utils import (
    smiles2vector_fg,
    smiles2vector_mfg,
    standardize_smiles,
)
from src.models.fgr_module import FGRPretrainLitModule

In [ ]:
model = FGRPretrainLitModule.load_from_checkpoint(
    "/home/rajeeva/Project/outputs/epoch_000_val_0.8505.ckpt"
)
model.eval();

In [ ]:
fgroups = pd.read_parquet("fg.parquet")["SMARTS"].tolist()  # Get functional groups
fgroups_list = [MolFromSmarts(x) for x in fgroups]  # Convert to RDKit Mol
tokenizer = Tokenizer.from_file(
    os.path.join(
        "tokenizers",
        f"BPE_pubchem_{500}.json",
    )
)  # Load tokenizer

In [ ]:
def get_representation(
    smiles: List[str],
    method: str,
    fgroups_list: List[MolFromSmarts],
    tokenizer: Tokenizer,
) -> np.ndarray:
    smiles = [standardize_smiles(smi) for smi in smiles]  # Standardize smiles
    if method == "FG":
        x = np.stack([smiles2vector_fg(x, fgroups_list) for x in smiles])
    elif method == "MFG":
        x = np.stack([smiles2vector_mfg(x, tokenizer) for x in smiles])
    elif method == "FGR":
        f_g = np.stack([smiles2vector_fg(x, fgroups_list) for x in smiles])
        mfg = np.stack([smiles2vector_mfg(x, tokenizer) for x in smiles])
        x = np.concatenate((f_g, mfg), axis=1)  # Concatenate both vectors
    else:
        raise ValueError("Method not supported")  # Raise error if method not supported
    return x

In [ ]:
x = get_representation(
    ["CC(C)(C)NCC(O)c1cc(Cl)c(N)c(c1)C(F)(F)F", "CCN", "CCF"], "FGR", fgroups_list, tokenizer
)
x = torch.tensor(x, dtype=torch.float32, device=model.device)
z_d = model(x)

In [ ]:
z_d[0].shape

In [ ]:
df = pd.read_csv("../../data/cancer/PRSIM_19Q4_latent_latest.csv")
df

In [ ]:
all_smiles = df["smiles_cleaned"].to_list()

In [ ]:
rep = get_representation(all_smiles, "FGR", fgroups_list, tokenizer)

In [ ]:
# rep = []
# correct_smiles = []
# error = []
# for i, smile in enumerate(all_smiles):
#     try:
#         rep.append(get_representation([smile], "FGR", fgroups_list, tokenizer))
#         correct_smiles.append(smile)
#     except:
#         error.append(smile)

In [ ]:
rep.shape

In [ ]:
# error_df = df[~df['smiles_cleaned'].isin(correct_smiles)]
# error_df

In [ ]:
len(all_smiles)

In [ ]:
latent_df = pd.DataFrame(data=rep, columns=[f"chem_{i}" for i in range(32672)])
# latent_df = pd.concat((df[['Condition', 'smiles_cleaned']].dropna(axis=0), latent_df),
#   ignore_index=True, axis =1)

In [ ]:
df[["Condition"]].join(latent_df).to_parquet(
    "../../data/cancer/ChemGroup.parquet", compression="zstd"
)

In [ ]:
latent_df["chem_1"].unique()

In [ ]:
import polars as pl

pl.read_parquet("../../data/cancer/ChemGroup.parquet").estimated_size("mb")